# 🎵 Audio AI: Text-to-Speech, Voice Processing a Generování Audia

**Autor:** Martin Švanda, Praut s.r.o.  
**Notebook:** 20/20 - Finální notebook série

---

## Co se naučíte

1. **Text-to-Speech (TTS)** - Generování řeči z textu pomocí moderních modelů
2. **Voice Cloning** - Klonování hlasu z krátkých ukázek
3. **Audio Processing** - Zpracování a analýza zvukových dat
4. **Music Generation** - Generování hudby pomocí AI
5. **Produkční Audio Pipeline** - End-to-end řešení

---

## 🔧 Instalace a Setup

In [ ]:
# Instalace potřebných knihoven
!pip install -q transformers torch torchaudio
!pip install -q datasets soundfile librosa scipy
!pip install -q accelerate sentencepiece

# Pro pokročilé TTS
!pip install -q TTS 2>/dev/null || echo "TTS install optional"

# Pro audio vizualizaci
!pip install -q matplotlib numpy

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import soundfile as sf
import io
import json
import warnings
from typing import Dict, List, Optional, Tuple, Any, Union
from dataclasses import dataclass, field
from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import Audio, display
warnings.filterwarnings('ignore')

# Detekce zařízení
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Používám zařízení: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Paměť: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

---

## 🗣️ Část 1: Text-to-Speech s SpeechT5

SpeechT5 je unified model pro speech-text úlohy od Microsoftu.

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset

class SpeechT5TTS:
    """Text-to-Speech engine založený na SpeechT5."""
    
    def __init__(self):
        """
        Inicializuje SpeechT5 TTS systém.
        """
        print("📥 Načítám SpeechT5 modely...")
        
        # Processor pro tokenizaci textu
        self.processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
        
        # TTS model
        self.model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
        self.model.to(device)
        self.model.eval()
        
        # Vocoder pro generování waveformu
        self.vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
        self.vocoder.to(device)
        self.vocoder.eval()
        
        # Načtení speaker embeddingů
        print("📥 Načítám speaker embeddingy...")
        embeddings_dataset = load_dataset(
            "Matthijs/cmu-arctic-xvectors", 
            split="validation"
        )
        self.speaker_embeddings = torch.tensor(
            embeddings_dataset[7306]["xvector"]
        ).unsqueeze(0).to(device)
        
        # Sample rate
        self.sample_rate = 16000
        
        print("✅ SpeechT5 TTS připraven")
    
    def synthesize(self, text: str, 
                   speaker_embedding: Optional[torch.Tensor] = None) -> np.ndarray:
        """
        Syntetizuje řeč z textu.
        
        Args:
            text: Text k syntéze
            speaker_embedding: Volitelný speaker embedding pro změnu hlasu
        
        Returns:
            Audio waveform jako numpy array
        """
        # Použití výchozího nebo poskytnutého embeddingu
        embedding = speaker_embedding if speaker_embedding is not None else self.speaker_embeddings
        
        # Tokenizace textu
        inputs = self.processor(text=text, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Generování mel spectrogramu
        with torch.no_grad():
            speech = self.model.generate_speech(
                inputs["input_ids"],
                embedding,
                vocoder=self.vocoder
            )
        
        return speech.cpu().numpy()
    
    def synthesize_long(self, text: str, 
                        max_chunk_length: int = 500) -> np.ndarray:
        """
        Syntetizuje delší text rozdělením na chunky.
        
        Args:
            text: Dlouhý text k syntéze
            max_chunk_length: Maximální délka chunku
        
        Returns:
            Spojený audio waveform
        """
        # Rozdělení na věty
        sentences = self._split_text(text)
        
        # Spojení do chunků
        chunks = []
        current_chunk = ""
        
        for sentence in sentences:
            if len(current_chunk) + len(sentence) <= max_chunk_length:
                current_chunk += sentence + " "
            else:
                if current_chunk:
                    chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
        
        if current_chunk:
            chunks.append(current_chunk.strip())
        
        # Syntéza jednotlivých chunků
        audio_parts = []
        for chunk in chunks:
            audio = self.synthesize(chunk)
            audio_parts.append(audio)
            # Krátká pauza mezi chunky
            pause = np.zeros(int(self.sample_rate * 0.3))
            audio_parts.append(pause)
        
        return np.concatenate(audio_parts)
    
    def _split_text(self, text: str) -> List[str]:
        """Rozdělí text na věty."""
        import re
        sentences = re.split(r'(?<=[.!?])\s+', text)
        return [s.strip() for s in sentences if s.strip()]
    
    def save_audio(self, audio: np.ndarray, filepath: str):
        """Uloží audio do souboru."""
        sf.write(filepath, audio, self.sample_rate)
        print(f"💾 Audio uloženo: {filepath}")
    
    def play_audio(self, audio: np.ndarray):
        """Přehraje audio v notebooku."""
        display(Audio(audio, rate=self.sample_rate))

print("\n" + "="*60)
print("SpeechT5TTS Engine")
print("="*60)

In [ ]:
# Inicializace TTS
tts_engine = SpeechT5TTS()

In [ ]:
# Test základní syntézy
test_text = "Hello, welcome to Praut AI solutions. We specialize in artificial intelligence and automation."

print(f"📝 Text: {test_text}")
print("🎤 Generuji audio...")

audio = tts_engine.synthesize(test_text)

print(f"✅ Audio vygenerováno: {len(audio)} samples ({len(audio)/tts_engine.sample_rate:.2f}s)")

# Vizualizace waveformu
plt.figure(figsize=(12, 4))
plt.plot(audio)
plt.title("Waveform syntetizovaného audia")
plt.xlabel("Samples")
plt.ylabel("Amplitude")
plt.tight_layout()
plt.show()

# Přehrání
tts_engine.play_audio(audio)

In [ ]:
# Test delšího textu
long_text = """
Artificial intelligence is transforming how businesses operate. 
At Praut, we help companies integrate AI into their workflows.
Our solutions include automated document processing, intelligent chatbots, and predictive analytics.
Contact us today to learn how AI can benefit your organization.
"""

print("📝 Dlouhý text:")
print(long_text[:100] + "...")
print("\n🎤 Generuji audio pro dlouhý text...")

long_audio = tts_engine.synthesize_long(long_text)

print(f"✅ Audio vygenerováno: {len(long_audio)/tts_engine.sample_rate:.2f}s")
tts_engine.play_audio(long_audio)

---

## 🎭 Část 2: BARK - Pokročilý TTS s Emocemi

BARK je generativní audio model od Suno AI, který podporuje emoce, smích, a neverbální zvuky.

In [ ]:
from transformers import AutoProcessor, BarkModel

class BarkTTS:
    """Pokročilý TTS s podporou emocí pomocí BARK modelu."""
    
    # Dostupné voice presety
    VOICE_PRESETS = {
        'male_1': 'v2/en_speaker_6',
        'male_2': 'v2/en_speaker_9',
        'female_1': 'v2/en_speaker_1',
        'female_2': 'v2/en_speaker_3',
        'narrator': 'v2/en_speaker_0',
    }
    
    # Speciální tagy pro emoce
    EMOTION_TAGS = {
        'laugh': '[laughter]',
        'sigh': '[sighs]',
        'gasp': '[gasps]',
        'clear_throat': '[clears throat]',
        'pause': '...',
        'music': '♪',
    }
    
    def __init__(self, model_name: str = "suno/bark-small"):
        """
        Args:
            model_name: Název modelu (bark-small nebo bark)
        """
        print(f"📥 Načítám BARK model: {model_name}")
        
        self.processor = AutoProcessor.from_pretrained(model_name)
        self.model = BarkModel.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
        self.model.to(device)
        self.model.eval()
        
        self.sample_rate = 24000  # BARK používá 24kHz
        
        print("✅ BARK model načten")
    
    def synthesize(self, text: str, 
                   voice_preset: str = 'narrator',
                   add_silence: bool = True) -> np.ndarray:
        """
        Syntetizuje řeč s možností emocí.
        
        Args:
            text: Text k syntéze (může obsahovat emotion tagy)
            voice_preset: Preset hlasu
            add_silence: Přidat ticho na konec
        
        Returns:
            Audio waveform
        """
        # Získání voice presetu
        preset = self.VOICE_PRESETS.get(voice_preset, voice_preset)
        
        # Příprava vstupů
        inputs = self.processor(text, voice_preset=preset)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Generování
        with torch.no_grad():
            audio_array = self.model.generate(**inputs)
        
        audio = audio_array.cpu().numpy().squeeze()
        
        # Přidání ticha na konec
        if add_silence:
            silence = np.zeros(int(self.sample_rate * 0.25))
            audio = np.concatenate([audio, silence])
        
        return audio
    
    def synthesize_with_emotion(self, text: str, 
                                emotion: str,
                                voice_preset: str = 'narrator') -> np.ndarray:
        """
        Syntetizuje text s přidanou emocí.
        
        Args:
            text: Text k syntéze
            emotion: Typ emoce (laugh, sigh, gasp, etc.)
            voice_preset: Preset hlasu
        
        Returns:
            Audio waveform
        """
        emotion_tag = self.EMOTION_TAGS.get(emotion, '')
        
        # Přidání emotion tagu
        if emotion_tag:
            enhanced_text = f"{text} {emotion_tag}"
        else:
            enhanced_text = text
        
        return self.synthesize(enhanced_text, voice_preset)
    
    def synthesize_dialogue(self, dialogue: List[Dict[str, str]]) -> np.ndarray:
        """
        Syntetizuje dialog mezi více mluvčími.
        
        Args:
            dialogue: Seznam replik [{"speaker": "male_1", "text": "..."}]
        
        Returns:
            Spojený audio waveform
        """
        audio_parts = []
        
        for turn in dialogue:
            speaker = turn.get('speaker', 'narrator')
            text = turn.get('text', '')
            emotion = turn.get('emotion', None)
            
            if emotion:
                audio = self.synthesize_with_emotion(text, emotion, speaker)
            else:
                audio = self.synthesize(text, speaker)
            
            audio_parts.append(audio)
            
            # Pauza mezi replikami
            pause = np.zeros(int(self.sample_rate * 0.5))
            audio_parts.append(pause)
        
        return np.concatenate(audio_parts)
    
    def play_audio(self, audio: np.ndarray):
        """Přehraje audio."""
        display(Audio(audio, rate=self.sample_rate))
    
    def list_voices(self):
        """Vypíše dostupné hlasy."""
        print("🎭 Dostupné hlasové presety:")
        for name, preset in self.VOICE_PRESETS.items():
            print(f"   {name}: {preset}")
        print("\n🎭 Dostupné emotion tagy:")
        for name, tag in self.EMOTION_TAGS.items():
            print(f"   {name}: {tag}")

print("\n" + "="*60)
print("BARK TTS Engine")
print("="*60)

In [ ]:
# Inicializace BARK
bark_tts = BarkTTS("suno/bark-small")
bark_tts.list_voices()

In [ ]:
# Test BARK s emocemi
print("\n🎭 Test BARK s různými hlasy a emocemi:")
print("-" * 50)

# Základní syntéza
text = "Welcome to Praut AI. We make artificial intelligence accessible for everyone."
print(f"📝 Text: {text}")

audio = bark_tts.synthesize(text, voice_preset='narrator')
print(f"✅ Audio vygenerováno ({len(audio)/bark_tts.sample_rate:.2f}s)")
bark_tts.play_audio(audio)

In [ ]:
# Test dialogu
dialogue = [
    {"speaker": "female_1", "text": "Have you tried the new AI automation system?"},
    {"speaker": "male_1", "text": "Yes! It saved us so much time.", "emotion": "laugh"},
    {"speaker": "female_1", "text": "That's amazing. I should implement it too."},
]

print("\n🎭 Generuji dialog:")
for turn in dialogue:
    print(f"   [{turn['speaker']}]: {turn['text']}")

dialogue_audio = bark_tts.synthesize_dialogue(dialogue)
print(f"\n✅ Dialog vygenerován ({len(dialogue_audio)/bark_tts.sample_rate:.2f}s)")
bark_tts.play_audio(dialogue_audio)

---

## 🎼 Část 3: Audio Processing a Analýza

Nástroje pro zpracování, analýzu a transformaci audio dat.

In [ ]:
import librosa
import librosa.display
from scipy import signal
from scipy.io import wavfile

class AudioProcessor:
    """Komplexní nástroj pro zpracování audia."""
    
    def __init__(self, sample_rate: int = 16000):
        """
        Args:
            sample_rate: Výchozí sample rate
        """
        self.sample_rate = sample_rate
    
    def load_audio(self, filepath: str, 
                   target_sr: Optional[int] = None) -> Tuple[np.ndarray, int]:
        """
        Načte audio soubor.
        
        Args:
            filepath: Cesta k souboru
            target_sr: Cílový sample rate (None = ponechat originál)
        
        Returns:
            Tuple (audio_data, sample_rate)
        """
        audio, sr = librosa.load(filepath, sr=target_sr)
        return audio, sr
    
    def save_audio(self, audio: np.ndarray, filepath: str, 
                   sample_rate: Optional[int] = None):
        """Uloží audio do souboru."""
        sr = sample_rate or self.sample_rate
        sf.write(filepath, audio, sr)
    
    def resample(self, audio: np.ndarray, 
                 orig_sr: int, target_sr: int) -> np.ndarray:
        """Převzorkuje audio na jiný sample rate."""
        return librosa.resample(audio, orig_sr=orig_sr, target_sr=target_sr)
    
    def normalize(self, audio: np.ndarray, 
                  target_db: float = -20.0) -> np.ndarray:
        """Normalizuje hlasitost audia."""
        # Výpočet aktuální RMS
        rms = np.sqrt(np.mean(audio**2))
        if rms == 0:
            return audio
        
        # Cílová RMS
        target_rms = 10 ** (target_db / 20)
        
        # Škálování
        return audio * (target_rms / rms)
    
    def trim_silence(self, audio: np.ndarray, 
                     top_db: int = 20) -> np.ndarray:
        """Ořízne ticho ze začátku a konce."""
        trimmed, _ = librosa.effects.trim(audio, top_db=top_db)
        return trimmed
    
    def apply_noise_reduction(self, audio: np.ndarray, 
                               noise_factor: float = 0.1) -> np.ndarray:
        """
        Jednoduchá redukce šumu pomocí spektrálního gatingu.
        
        Args:
            audio: Vstupní audio
            noise_factor: Faktor pro práh šumu (0-1)
        
        Returns:
            Audio s redukovaným šumem
        """
        # STFT
        stft = librosa.stft(audio)
        magnitude = np.abs(stft)
        phase = np.angle(stft)
        
        # Odhad šumového prahu
        noise_threshold = np.mean(magnitude) * noise_factor
        
        # Spektrální gating
        magnitude_cleaned = np.where(
            magnitude > noise_threshold, 
            magnitude, 
            magnitude * 0.1
        )
        
        # Rekonstrukce
        stft_cleaned = magnitude_cleaned * np.exp(1j * phase)
        audio_cleaned = librosa.istft(stft_cleaned)
        
        return audio_cleaned
    
    def change_speed(self, audio: np.ndarray, 
                     speed_factor: float) -> np.ndarray:
        """
        Změní rychlost audia bez změny výšky.
        
        Args:
            audio: Vstupní audio
            speed_factor: Faktor rychlosti (>1 = rychlejší, <1 = pomalejší)
        
        Returns:
            Audio s upravenou rychlostí
        """
        return librosa.effects.time_stretch(audio, rate=speed_factor)
    
    def change_pitch(self, audio: np.ndarray, 
                     semitones: float,
                     sample_rate: Optional[int] = None) -> np.ndarray:
        """
        Změní výšku tónu audia.
        
        Args:
            audio: Vstupní audio
            semitones: Počet půltónů (+/- pro zvýšení/snížení)
            sample_rate: Sample rate audia
        
        Returns:
            Audio s upravenou výškou
        """
        sr = sample_rate or self.sample_rate
        return librosa.effects.pitch_shift(audio, sr=sr, n_steps=semitones)
    
    def extract_features(self, audio: np.ndarray,
                         sample_rate: Optional[int] = None) -> Dict[str, Any]:
        """
        Extrahuje audio features.
        
        Returns:
            Dict s různými audio features
        """
        sr = sample_rate or self.sample_rate
        
        features = {}
        
        # Základní statistiky
        features['duration'] = len(audio) / sr
        features['rms'] = float(np.sqrt(np.mean(audio**2)))
        features['zero_crossing_rate'] = float(np.mean(librosa.feature.zero_crossing_rate(audio)))
        
        # Spektrální features
        spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
        features['spectral_centroid_mean'] = float(np.mean(spectral_centroids))
        features['spectral_centroid_std'] = float(np.std(spectral_centroids))
        
        # Spectral bandwidth
        spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
        features['spectral_bandwidth_mean'] = float(np.mean(spectral_bandwidth))
        
        # MFCCs
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        features['mfcc_means'] = [float(np.mean(mfcc)) for mfcc in mfccs]
        
        # Chroma features
        chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
        features['chroma_mean'] = float(np.mean(chroma))
        
        # Tempo
        try:
            tempo, _ = librosa.beat.beat_track(y=audio, sr=sr)
            features['tempo'] = float(tempo)
        except:
            features['tempo'] = None
        
        return features
    
    def visualize(self, audio: np.ndarray, 
                  sample_rate: Optional[int] = None,
                  title: str = "Audio Analysis"):
        """
        Vizualizuje audio (waveform, spectrogram, mel spectrogram).
        """
        sr = sample_rate or self.sample_rate
        
        fig, axes = plt.subplots(3, 1, figsize=(14, 10))
        
        # Waveform
        librosa.display.waveshow(audio, sr=sr, ax=axes[0])
        axes[0].set_title(f"{title} - Waveform")
        axes[0].set_xlabel("Time (s)")
        axes[0].set_ylabel("Amplitude")
        
        # Spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(audio)), ref=np.max)
        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz', ax=axes[1])
        axes[1].set_title(f"{title} - Spectrogram")
        axes[1].set_xlabel("Time (s)")
        axes[1].set_ylabel("Frequency (Hz)")
        
        # Mel Spectrogram
        mel_spec = librosa.feature.melspectrogram(y=audio, sr=sr)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        librosa.display.specshow(mel_spec_db, sr=sr, x_axis='time', y_axis='mel', ax=axes[2])
        axes[2].set_title(f"{title} - Mel Spectrogram")
        axes[2].set_xlabel("Time (s)")
        axes[2].set_ylabel("Mel Frequency")
        
        plt.tight_layout()
        plt.show()

# Inicializace
audio_processor = AudioProcessor(sample_rate=16000)

print("\n" + "="*60)
print("AudioProcessor připraven")
print("="*60)

In [ ]:
# Použití audio processoru na syntetizované audio
print("\n🔊 Analýza syntetizovaného audia:")
print("-" * 50)

# Použijeme audio z TTS
test_audio = tts_engine.synthesize("Hello, this is a test of audio processing capabilities.")

# Extrakce features
features = audio_processor.extract_features(test_audio, tts_engine.sample_rate)

print("\n📊 Audio Features:")
print(f"   Délka: {features['duration']:.2f}s")
print(f"   RMS: {features['rms']:.4f}")
print(f"   Zero Crossing Rate: {features['zero_crossing_rate']:.4f}")
print(f"   Spectral Centroid: {features['spectral_centroid_mean']:.1f} Hz")
print(f"   Spectral Bandwidth: {features['spectral_bandwidth_mean']:.1f} Hz")
if features['tempo']:
    print(f"   Tempo: {features['tempo']:.1f} BPM")

# Vizualizace
audio_processor.visualize(test_audio, tts_engine.sample_rate, "TTS Audio")

In [ ]:
# Demonstrace audio transformací
print("\n🔄 Demonstrace audio transformací:")
print("-" * 50)

# Původní audio
original_audio = test_audio.copy()
sr = tts_engine.sample_rate

# 1. Změna rychlosti
faster_audio = audio_processor.change_speed(original_audio, speed_factor=1.5)
slower_audio = audio_processor.change_speed(original_audio, speed_factor=0.75)

print(f"\n▶️ Rychlejší verze (1.5x): {len(faster_audio)/sr:.2f}s")
display(Audio(faster_audio, rate=sr))

print(f"\n⏸️ Pomalejší verze (0.75x): {len(slower_audio)/sr:.2f}s")
display(Audio(slower_audio, rate=sr))

In [ ]:
# 2. Změna výšky
higher_pitch = audio_processor.change_pitch(original_audio, semitones=4, sample_rate=sr)
lower_pitch = audio_processor.change_pitch(original_audio, semitones=-4, sample_rate=sr)

print(f"\n🔼 Vyšší hlas (+4 půltóny):")
display(Audio(higher_pitch, rate=sr))

print(f"\n🔽 Nižší hlas (-4 půltóny):")
display(Audio(lower_pitch, rate=sr))

---

## 🎵 Část 4: MusicGen - Generování Hudby

MusicGen od Meta AI generuje hudbu z textových popisů.

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

class MusicGenerator:
    """Generátor hudby pomocí MusicGen modelu."""
    
    def __init__(self, model_name: str = "facebook/musicgen-small"):
        """
        Args:
            model_name: Název modelu (small/medium/large)
        """
        print(f"📥 Načítám MusicGen model: {model_name}")
        
        self.processor = AutoProcessor.from_pretrained(model_name)
        self.model = MusicgenForConditionalGeneration.from_pretrained(model_name)
        self.model.to(device)
        self.model.eval()
        
        self.sample_rate = self.model.config.audio_encoder.sampling_rate
        
        print(f"✅ MusicGen načten (sample rate: {self.sample_rate} Hz)")
    
    def generate(self, prompt: str, 
                 duration_seconds: float = 8.0,
                 guidance_scale: float = 3.0) -> np.ndarray:
        """
        Generuje hudbu z textového popisu.
        
        Args:
            prompt: Textový popis požadované hudby
            duration_seconds: Délka v sekundách
            guidance_scale: Síla vedení textem
        
        Returns:
            Audio waveform
        """
        # Příprava vstupů
        inputs = self.processor(
            text=[prompt],
            padding=True,
            return_tensors="pt"
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Výpočet max_new_tokens pro požadovanou délku
        # MusicGen generuje ~50 tokenů za sekundu
        max_new_tokens = int(duration_seconds * 50)
        
        # Generování
        with torch.no_grad():
            audio_values = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                guidance_scale=guidance_scale,
                do_sample=True
            )
        
        # Konverze na numpy
        audio = audio_values[0, 0].cpu().numpy()
        
        return audio
    
    def generate_variations(self, prompt: str, 
                            num_variations: int = 3,
                            duration_seconds: float = 5.0) -> List[np.ndarray]:
        """
        Generuje více variací hudby ze stejného promptu.
        
        Args:
            prompt: Textový popis
            num_variations: Počet variací
            duration_seconds: Délka každé variace
        
        Returns:
            Seznam audio waveformů
        """
        variations = []
        
        for i in range(num_variations):
            print(f"   Generuji variaci {i+1}/{num_variations}...")
            audio = self.generate(prompt, duration_seconds)
            variations.append(audio)
        
        return variations
    
    def concatenate_sections(self, sections: List[Tuple[str, float]],
                             crossfade_duration: float = 0.5) -> np.ndarray:
        """
        Generuje a spojuje více sekcí hudby.
        
        Args:
            sections: Seznam (prompt, duration) pro každou sekci
            crossfade_duration: Délka crossfade mezi sekcemi
        
        Returns:
            Spojené audio
        """
        audio_sections = []
        
        for i, (prompt, duration) in enumerate(sections):
            print(f"   Generuji sekci {i+1}: '{prompt[:30]}...'")
            audio = self.generate(prompt, duration)
            audio_sections.append(audio)
        
        # Spojení s crossfade
        if len(audio_sections) == 1:
            return audio_sections[0]
        
        crossfade_samples = int(crossfade_duration * self.sample_rate)
        result = audio_sections[0]
        
        for section in audio_sections[1:]:
            # Crossfade
            fade_out = np.linspace(1, 0, crossfade_samples)
            fade_in = np.linspace(0, 1, crossfade_samples)
            
            # Aplikace fade
            result[-crossfade_samples:] *= fade_out
            section[:crossfade_samples] *= fade_in
            
            # Spojení s překryvem
            result[-crossfade_samples:] += section[:crossfade_samples]
            result = np.concatenate([result, section[crossfade_samples:]])
        
        return result
    
    def play_audio(self, audio: np.ndarray):
        """Přehraje audio."""
        display(Audio(audio, rate=self.sample_rate))
    
    def save_audio(self, audio: np.ndarray, filepath: str):
        """Uloží audio do souboru."""
        sf.write(filepath, audio, self.sample_rate)
        print(f"💾 Audio uloženo: {filepath}")

print("\n" + "="*60)
print("MusicGenerator Engine")
print("="*60)

In [ ]:
# Inicializace MusicGen
music_gen = MusicGenerator("facebook/musicgen-small")

In [ ]:
# Generování hudby z textového popisu
print("\n🎵 Generování hudby z textového popisu:")
print("-" * 50)

prompts = [
    "Upbeat corporate background music with positive energy",
    "Calm ambient electronic music for focus and productivity",
    "Energetic rock guitar riff with drums",
]

for prompt in prompts:
    print(f"\n🎼 Prompt: '{prompt}'")
    print("   Generuji...")
    
    audio = music_gen.generate(prompt, duration_seconds=5.0)
    
    print(f"   ✅ Vygenerováno: {len(audio)/music_gen.sample_rate:.2f}s")
    music_gen.play_audio(audio)
    print()

---

## 🏭 Část 5: Produkční Audio Pipeline

Kompletní pipeline pro produkční audio aplikace.

In [ ]:
import time
from enum import Enum
from datetime import datetime
import hashlib

class AudioTaskType(Enum):
    TTS = "text_to_speech"
    MUSIC = "music_generation"
    PROCESSING = "audio_processing"
    ANALYSIS = "audio_analysis"

@dataclass
class AudioJob:
    """Reprezentace audio úlohy."""
    job_id: str
    task_type: AudioTaskType
    input_data: Dict[str, Any]
    output_audio: Optional[np.ndarray] = None
    output_path: Optional[str] = None
    metadata: Dict[str, Any] = field(default_factory=dict)
    status: str = "pending"
    error: Optional[str] = None
    created_at: datetime = field(default_factory=datetime.now)
    completed_at: Optional[datetime] = None
    processing_time: float = 0.0

class ProductionAudioPipeline:
    """
    Produkční pipeline pro komplexní audio zpracování.
    
    Kombinuje TTS, hudební generování a audio processing
    do jednoho unifikovaného rozhraní.
    """
    
    def __init__(self,
                 enable_tts: bool = True,
                 enable_music: bool = True,
                 cache_size: int = 50):
        """
        Args:
            enable_tts: Povolit TTS komponenty
            enable_music: Povolit generování hudby
            cache_size: Velikost cache
        """
        print("🏭 Inicializace Production Audio Pipeline...")
        
        self.components = {}
        
        # TTS komponenty
        if enable_tts:
            print("   📢 Inicializace TTS...")
            self.components['tts_speecht5'] = SpeechT5TTS()
            # BARK je volitelný kvůli velikosti
            try:
                self.components['tts_bark'] = BarkTTS("suno/bark-small")
            except Exception as e:
                print(f"   ⚠️ BARK nedostupný: {e}")
        
        # Music generování
        if enable_music:
            print("   🎵 Inicializace MusicGen...")
            try:
                self.components['music'] = MusicGenerator("facebook/musicgen-small")
            except Exception as e:
                print(f"   ⚠️ MusicGen nedostupný: {e}")
        
        # Audio processor je vždy dostupný
        self.components['processor'] = AudioProcessor()
        
        # Cache a statistiky
        self.cache = {}
        self.cache_size = cache_size
        self.jobs_history = []
        self.stats = {
            'total_jobs': 0,
            'successful_jobs': 0,
            'failed_jobs': 0,
            'total_audio_generated_seconds': 0,
            'cache_hits': 0,
            'by_type': defaultdict(int)
        }
        
        print("\n✅ Pipeline inicializován")
        print(f"   Komponenty: {list(self.components.keys())}")
    
    def text_to_speech(self, text: str,
                       engine: str = "speecht5",
                       voice: str = "default",
                       **kwargs) -> AudioJob:
        """
        Převede text na řeč.
        
        Args:
            text: Text k syntéze
            engine: TTS engine (speecht5/bark)
            voice: Hlas pro syntézu
            **kwargs: Další parametry pro engine
        
        Returns:
            AudioJob s výsledkem
        """
        job = self._create_job(AudioTaskType.TTS, {
            'text': text,
            'engine': engine,
            'voice': voice,
            **kwargs
        })
        
        try:
            start_time = time.time()
            
            # Výběr engine
            if engine == "bark" and 'tts_bark' in self.components:
                tts = self.components['tts_bark']
                audio = tts.synthesize(text, voice_preset=voice)
                sample_rate = tts.sample_rate
            else:
                tts = self.components.get('tts_speecht5')
                if tts is None:
                    raise RuntimeError("TTS komponenta není dostupná")
                audio = tts.synthesize_long(text) if len(text) > 200 else tts.synthesize(text)
                sample_rate = tts.sample_rate
            
            job.output_audio = audio
            job.metadata['sample_rate'] = sample_rate
            job.metadata['duration'] = len(audio) / sample_rate
            job.status = "completed"
            job.processing_time = time.time() - start_time
            job.completed_at = datetime.now()
            
            self._update_stats(job, success=True)
            
        except Exception as e:
            job.status = "failed"
            job.error = str(e)
            self._update_stats(job, success=False)
        
        self.jobs_history.append(job)
        return job
    
    def generate_music(self, prompt: str,
                       duration: float = 8.0,
                       **kwargs) -> AudioJob:
        """
        Generuje hudbu z textového popisu.
        
        Args:
            prompt: Textový popis hudby
            duration: Délka v sekundách
            **kwargs: Další parametry
        
        Returns:
            AudioJob s výsledkem
        """
        job = self._create_job(AudioTaskType.MUSIC, {
            'prompt': prompt,
            'duration': duration,
            **kwargs
        })
        
        try:
            if 'music' not in self.components:
                raise RuntimeError("MusicGen komponenta není dostupná")
            
            start_time = time.time()
            
            music_gen = self.components['music']
            audio = music_gen.generate(prompt, duration)
            
            job.output_audio = audio
            job.metadata['sample_rate'] = music_gen.sample_rate
            job.metadata['duration'] = len(audio) / music_gen.sample_rate
            job.status = "completed"
            job.processing_time = time.time() - start_time
            job.completed_at = datetime.now()
            
            self._update_stats(job, success=True)
            
        except Exception as e:
            job.status = "failed"
            job.error = str(e)
            self._update_stats(job, success=False)
        
        self.jobs_history.append(job)
        return job
    
    def process_audio(self, audio: np.ndarray,
                      sample_rate: int,
                      operations: List[Dict[str, Any]]) -> AudioJob:
        """
        Aplikuje řetězec operací na audio.
        
        Args:
            audio: Vstupní audio
            sample_rate: Sample rate
            operations: Seznam operací [{"type": "normalize", "params": {...}}]
        
        Returns:
            AudioJob s výsledkem
        """
        job = self._create_job(AudioTaskType.PROCESSING, {
            'audio_length': len(audio),
            'sample_rate': sample_rate,
            'operations': operations
        })
        
        try:
            start_time = time.time()
            processor = self.components['processor']
            
            processed_audio = audio.copy()
            applied_operations = []
            
            for op in operations:
                op_type = op.get('type')
                params = op.get('params', {})
                
                if op_type == 'normalize':
                    processed_audio = processor.normalize(processed_audio, **params)
                elif op_type == 'trim_silence':
                    processed_audio = processor.trim_silence(processed_audio, **params)
                elif op_type == 'noise_reduction':
                    processed_audio = processor.apply_noise_reduction(processed_audio, **params)
                elif op_type == 'change_speed':
                    processed_audio = processor.change_speed(processed_audio, **params)
                elif op_type == 'change_pitch':
                    processed_audio = processor.change_pitch(
                        processed_audio, sample_rate=sample_rate, **params
                    )
                elif op_type == 'resample':
                    target_sr = params.get('target_sr', 16000)
                    processed_audio = processor.resample(
                        processed_audio, sample_rate, target_sr
                    )
                    sample_rate = target_sr
                
                applied_operations.append(op_type)
            
            job.output_audio = processed_audio
            job.metadata['sample_rate'] = sample_rate
            job.metadata['duration'] = len(processed_audio) / sample_rate
            job.metadata['applied_operations'] = applied_operations
            job.status = "completed"
            job.processing_time = time.time() - start_time
            job.completed_at = datetime.now()
            
            self._update_stats(job, success=True)
            
        except Exception as e:
            job.status = "failed"
            job.error = str(e)
            self._update_stats(job, success=False)
        
        self.jobs_history.append(job)
        return job
    
    def analyze_audio(self, audio: np.ndarray,
                      sample_rate: int) -> AudioJob:
        """
        Analyzuje audio a extrahuje features.
        
        Args:
            audio: Vstupní audio
            sample_rate: Sample rate
        
        Returns:
            AudioJob s analýzou v metadata
        """
        job = self._create_job(AudioTaskType.ANALYSIS, {
            'audio_length': len(audio),
            'sample_rate': sample_rate
        })
        
        try:
            start_time = time.time()
            processor = self.components['processor']
            
            features = processor.extract_features(audio, sample_rate)
            
            job.metadata['features'] = features
            job.metadata['sample_rate'] = sample_rate
            job.status = "completed"
            job.processing_time = time.time() - start_time
            job.completed_at = datetime.now()
            
            self._update_stats(job, success=True)
            
        except Exception as e:
            job.status = "failed"
            job.error = str(e)
            self._update_stats(job, success=False)
        
        self.jobs_history.append(job)
        return job
    
    def _create_job(self, task_type: AudioTaskType, 
                    input_data: Dict) -> AudioJob:
        """Vytvoří novou úlohu."""
        job_id = hashlib.md5(
            f"{task_type.value}_{datetime.now().isoformat()}".encode()
        ).hexdigest()[:12]
        
        return AudioJob(
            job_id=job_id,
            task_type=task_type,
            input_data=input_data
        )
    
    def _update_stats(self, job: AudioJob, success: bool):
        """Aktualizuje statistiky."""
        self.stats['total_jobs'] += 1
        self.stats['by_type'][job.task_type.value] += 1
        
        if success:
            self.stats['successful_jobs'] += 1
            if job.metadata.get('duration'):
                self.stats['total_audio_generated_seconds'] += job.metadata['duration']
        else:
            self.stats['failed_jobs'] += 1
    
    def get_stats(self) -> Dict:
        """Vrátí statistiky pipeline."""
        stats = dict(self.stats)
        stats['by_type'] = dict(stats['by_type'])
        
        if stats['total_jobs'] > 0:
            stats['success_rate'] = stats['successful_jobs'] / stats['total_jobs']
        
        return stats
    
    def get_recent_jobs(self, n: int = 10) -> List[AudioJob]:
        """Vrátí posledních N úloh."""
        return self.jobs_history[-n:]
    
    def play_job_audio(self, job: AudioJob):
        """Přehraje audio z úlohy."""
        if job.output_audio is not None and job.metadata.get('sample_rate'):
            display(Audio(job.output_audio, rate=job.metadata['sample_rate']))
        else:
            print("⚠️ Úloha nemá výstupní audio")

print("\n" + "="*60)
print("ProductionAudioPipeline připraven")
print("="*60)

In [ ]:
# Inicializace produkčního pipeline
audio_pipeline = ProductionAudioPipeline(
    enable_tts=True,
    enable_music=True,
    cache_size=50
)

In [ ]:
# Test TTS
print("\n" + "="*60)
print("TEST: Text-to-Speech")
print("="*60)

tts_job = audio_pipeline.text_to_speech(
    text="Welcome to Praut AI Pipeline. This is a demonstration of our text to speech capabilities.",
    engine="speecht5"
)

print(f"\n📋 Job ID: {tts_job.job_id}")
print(f"   Status: {tts_job.status}")
print(f"   Délka: {tts_job.metadata.get('duration', 0):.2f}s")
print(f"   Čas zpracování: {tts_job.processing_time:.2f}s")

if tts_job.status == "completed":
    audio_pipeline.play_job_audio(tts_job)

In [ ]:
# Test audio processingu
print("\n" + "="*60)
print("TEST: Audio Processing")
print("="*60)

# Použijeme audio z TTS
if tts_job.output_audio is not None:
    processing_job = audio_pipeline.process_audio(
        audio=tts_job.output_audio,
        sample_rate=tts_job.metadata['sample_rate'],
        operations=[
            {"type": "normalize", "params": {"target_db": -18}},
            {"type": "trim_silence", "params": {"top_db": 25}},
            {"type": "change_pitch", "params": {"semitones": -2}},
        ]
    )
    
    print(f"\n📋 Job ID: {processing_job.job_id}")
    print(f"   Status: {processing_job.status}")
    print(f"   Operace: {processing_job.metadata.get('applied_operations', [])}")
    print(f"   Výsledná délka: {processing_job.metadata.get('duration', 0):.2f}s")
    
    if processing_job.status == "completed":
        print("\n🔊 Zpracované audio:")
        audio_pipeline.play_job_audio(processing_job)

In [ ]:
# Test generování hudby
print("\n" + "="*60)
print("TEST: Music Generation")
print("="*60)

music_job = audio_pipeline.generate_music(
    prompt="Uplifting corporate background music with piano and soft drums",
    duration=5.0
)

print(f"\n📋 Job ID: {music_job.job_id}")
print(f"   Status: {music_job.status}")
if music_job.status == "completed":
    print(f"   Délka: {music_job.metadata.get('duration', 0):.2f}s")
    print(f"   Čas zpracování: {music_job.processing_time:.2f}s")
    audio_pipeline.play_job_audio(music_job)
else:
    print(f"   Chyba: {music_job.error}")

In [ ]:
# Test analýzy
print("\n" + "="*60)
print("TEST: Audio Analysis")
print("="*60)

if tts_job.output_audio is not None:
    analysis_job = audio_pipeline.analyze_audio(
        audio=tts_job.output_audio,
        sample_rate=tts_job.metadata['sample_rate']
    )
    
    print(f"\n📋 Job ID: {analysis_job.job_id}")
    print(f"   Status: {analysis_job.status}")
    
    if analysis_job.status == "completed":
        features = analysis_job.metadata.get('features', {})
        print(f"\n📊 Extrahované features:")
        print(f"   Duration: {features.get('duration', 0):.2f}s")
        print(f"   RMS: {features.get('rms', 0):.4f}")
        print(f"   Spectral Centroid: {features.get('spectral_centroid_mean', 0):.1f} Hz")
        print(f"   Zero Crossing Rate: {features.get('zero_crossing_rate', 0):.4f}")

In [ ]:
# Finální statistiky pipeline
print("\n" + "="*60)
print("STATISTIKY PIPELINE")
print("="*60)

stats = audio_pipeline.get_stats()

print(f"\n📊 Celkové statistiky:")
print(f"   Celkem úloh: {stats['total_jobs']}")
print(f"   Úspěšných: {stats['successful_jobs']}")
print(f"   Neúspěšných: {stats['failed_jobs']}")
print(f"   Success rate: {stats.get('success_rate', 0):.1%}")
print(f"   Celkem vygenerováno audia: {stats['total_audio_generated_seconds']:.1f}s")

print(f"\n📈 Podle typu úlohy:")
for task_type, count in stats['by_type'].items():
    print(f"   {task_type}: {count}")

print(f"\n📜 Poslední úlohy:")
for job in audio_pipeline.get_recent_jobs(5):
    status_icon = "✅" if job.status == "completed" else "❌"
    print(f"   {status_icon} {job.job_id}: {job.task_type.value} ({job.processing_time:.2f}s)")

---

## 🎯 Shrnutí Série

Gratulujeme! Dokončili jste celou sérii **20 notebooků** o Hugging Face Transformers!

### Co jsme se naučili v tomto notebooku

| Komponenta | Model | Použití |
|------------|-------|--------|
| **SpeechT5** | microsoft/speecht5_tts | Text-to-Speech syntéza |
| **BARK** | suno/bark-small | TTS s emocemi a efekty |
| **MusicGen** | facebook/musicgen-small | Generování hudby z textu |
| **Librosa** | - | Audio processing a analýza |

### Přehled celé série

| # | Téma | Klíčové modely |
|---|------|---------------|
| 1 | Úvod do HF | Pipeline API, AutoModel |
| 2 | Klasifikace a NER | BERT, DistilBERT |
| 3 | Sentiment a emoce | RoBERTa, GoEmotions |
| 4 | Sumarizace a generování | BART, T5, GPT-2 |
| 5 | Překlad | MarianMT, mBART |
| 6 | Question Answering | BERT QA, Retrieval |
| 7 | Speech-to-Text | Whisper |
| 8-10 | Computer Vision | ViT, DETR, Segmentation |
| 11 | Embeddings a Search | Sentence Transformers |
| 12 | Fine-tuning | LoRA, PEFT |
| 13 | RAG systémy | Retrieval + Generation |
| 14 | LLM optimalizace | Quantization, vLLM |
| 15 | Multimodální modely | CLIP, LLaVA, BLIP |
| 16 | Time Series | Transformer forecasting |
| 17 | Doporučovací systémy | Collaborative filtering |
| 18 | Detekce anomálií | Autoencoder, Isolation Forest |
| 19 | Document AI | TrOCR, LayoutLM, Donut |
| **20** | **Audio AI** | **SpeechT5, BARK, MusicGen** |

In [ ]:
print("\n" + "="*60)
print("🎉 SÉRIE DOKONČENA!")
print("="*60)
print("\n🏆 Gratulujeme k dokončení všech 20 notebooků!")
print("\n📚 Naučili jste se:")
print("   ✅ Text processing (klasifikace, NER, sentiment)")
print("   ✅ Generování textu (sumarizace, překlad, QA)")
print("   ✅ Computer Vision (klasifikace, detekce, segmentace)")
print("   ✅ Speech (STT, TTS, voice cloning)")
print("   ✅ Multimodální AI (CLIP, LLaVA, BLIP)")
print("   ✅ Pokročilé techniky (RAG, fine-tuning, optimalizace)")
print("   ✅ Specializované aplikace (time series, recsys, anomaly)")
print("   ✅ Document AI (OCR, layout, extrakce)")
print("   ✅ Audio AI (TTS, hudba, zpracování)")
print("\n🚀 Další kroky:")
print("   • Aplikujte naučené na vlastní projekty")
print("   • Experimentujte s fine-tuningem na vlastních datech")
print("   • Sledujte novinky na Hugging Face Hub")
print("   • Přispějte do open-source komunity")
print("\n💼 Pro business konzultace: info@praut.cz")
print("\n" + "="*60)